In [1]:
from selenium import webdriver
import os

from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options


In [ ]:
def write_csv(bag):
    import csv
    import random
    import pandas as pd
    try:os.mkdir('comments_youtube/')
    except:pass
    df = pd.DataFrame.from_dict(bag, orient='index').T.to_csv('comments_youtube/'+str(random.randint(1,10000))+'.csv',index=False)
    print('CSV IS WRITTEN')

In [3]:
def show_more_button_clicker(driver,wait=15):
    while True:
        wait = WebDriverWait(driver,wait)
        try:
            wait.until(EC.visibility_of_element_located((By.XPATH,"//button[@class='yt-uix-button yt-uix-button-size-default yt-uix-button-default load-more-button yt-uix-load-more comment-section-renderer-paginator yt-uix-sessionlink']")))
            button = driver.find_element_by_xpath("//button[@class='yt-uix-button yt-uix-button-size-default yt-uix-button-default load-more-button yt-uix-load-more comment-section-renderer-paginator yt-uix-sessionlink']")
            button.click()
        except:
            print('No Show More button..')
            return False

In [4]:
def tag_text_validation(driver):
    try:
        driver.find_elements_by_tag_name('ytd-comment-thread-renderer')[-1:].pop()
        return True
    except:
        return False

In [5]:
def css_text_validation(driver):
    try:
        driver.find_elements_by_css_selector('.comment-thread-renderer.vve-check-visible.vve-check-hidden')[-1:].pop()
        return True
    except:
        return False

In [6]:
def text_validation(driver,css,tag):
    if css==True:
        css = driver.find_elements_by_css_selector('.comment-thread-renderer.vve-check-visible.vve-check-hidden')[-1:].pop()
        return css
    elif tag==True:
        tag = driver.find_elements_by_tag_name('ytd-comment-thread-renderer')[-1:].pop()
        return tag
    else:
        print('Problem in text_validation function, css or tag are not true!')
        exit()

In [7]:
def comment_extractor(driver,TAG=None,CSS=None,POP=None,i=None):
    if POP==True:
        if TAG==True:
            obj = driver.find_elements_by_tag_name('ytd-comment-thread-renderer')
            tag = obj[-1:].pop()
            ncomment = len(obj)
            return tag,ncomment
        elif CSS==True:
            obj = driver.find_elements_by_css_selector('.comment-thread-renderer.vve-check-visible.vve-check-hidden')
            css = obj[-1:].pop()
            ncomment = len(obj)
            return css,ncomment
        else:
            print('CSS or TAG ?')
            exit()
    
    elif POP==None: # If pop wasnt specified...
        if TAG==True:
            tag = driver.find_elements_by_tag_name('ytd-comment-thread-renderer')[i].text
            return tag
        elif CSS==True:
            css = driver.find_elements_by_css_selector('.comment-thread-renderer.vve-check-visible.vve-check-hidden')[i].text
            return css
        else:
            print('CSS text or TAG text?')
            exit()
    else:
        print('Problem in comment_extractor \n Pop is not True or None!')
        exit()

In [8]:
def video_iter(driver,url,pagetag=None):
    driver.get(url)
    if pagetag:
        # Click on videos
        for i in range(0,100):
            video = driver.find_elements_by_id(pagetag)[i]
            video.click()
            # ... do some stuff ...
            chrome.back()
    else:
        # click on video to stop it, so that it wouldnt pass to next video
        time.sleep(1)
        #driver.find_element_by_css_selector('.video-stream.html5-main-video').click()  # Class_name doesnt support compound names
        #driver.find_element_by_class_name('html5-video-container').click()
        #driver.find_element_by_css_selector('.ytp-play-button.ytp-button').click()
        # If 'Show More' Button Appears, Click on it.
        show_more_button_clicker(driver,6)
        
        # In case the comments doesn't show up at first.
        driver.find_element_by_tag_name('html').send_keys(Keys.END)
        time.sleep(2)
        driver.find_element_by_tag_name('html').send_keys(Keys.END)
        time.sleep(2)
        driver.find_element_by_tag_name('html').send_keys(Keys.END)
        time.sleep(2)
        
        # Comments can be extracted by whether css or tag method (because of the system of youtube)
        css = css_text_validation(driver)
        tag = tag_text_validation(driver)
        
        # Go down over comments, if last comment is the same, then you are at the bottom.
        last_comment = None
        done=False
        while done==False:
            driver.find_element_by_tag_name('html').send_keys(Keys.END)
            wait = WebDriverWait(driver,8)
            if css:
                wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,'.comment-thread-renderer.vve-check-visible.vve-check-hidden')))
            elif tag:
                wait.until(EC.presence_of_all_elements_located((By.TAG_NAME,'ytd-comment-thread-renderer')))
            else:
                print('css and tag are not True')
                exit()
            Comment = comment_extractor(driver,CSS=css,TAG=tag,POP=True)  # returns a text from css or tag 
            time.sleep(5)
            if Comment==last_comment:
                done=True
            else:
                last_comment=Comment
                pass
        # Iterate over the comments
        storage=0
        name,date,comment = list(),list(),list()
        ncomment = comment_extractor(driver,CSS=css,TAG=tag,POP=True)[1]
        print(ncomment)
        while True:
            for i in range(4,ncomment):
                print("i : {}".format(i))
                text = comment_extractor(driver=driver,TAG=tag,CSS=css,i=i)
                name.append(text.split('\n')[0])
                date.append(text.split('\n')[1])
                comment.append(text.split('\n')[2:])
                # Loop Breaking
                storage+=1
                if storage==ncomment-4:
                    print('Finished.')
                    driver.quit()
                    return name,date,comment
                else:
                    print("Number of comments stored: {}".format(storage))
                    #print(name)

In [41]:
path_to_extension = r"C:\Users\dogus\Dropbox\DgsPy_DBOX\webdrivers\1.16.14_0"
path = "C:/Users/dogus/Dropbox/DgsPy_DBOX/webdrivers/chromedriver.exe"

chrome_options = Options()
chrome_options.add_argument('load-extension=' + path_to_extension)
chrome_options.add_argument("--headless")

driver = webdriver.Chrome(executable_path=path,chrome_options=chrome_options)
driver.create_options()

#url = https://www.youtube.com/watch?v=1ZNrst0sKuE
#driver.get(url)

In [42]:
url ="https://www.youtube.com/watch?v=AccyIlzp_t0"
name,date,comment = video_iter(driver=driver,url=url)

dictionary = {
    'Name':name,
    'Date':date,
    'Comment':comment
}

write_csv(dictionary)

No button..
1418
i : 4
Number of comments stored: 1
i : 5
Number of comments stored: 2
i : 6
Number of comments stored: 3
i : 7
Number of comments stored: 4
i : 8
Number of comments stored: 5
i : 9
Number of comments stored: 6
i : 10
Number of comments stored: 7
i : 11
Number of comments stored: 8
i : 12
Number of comments stored: 9
i : 13
Number of comments stored: 10
i : 14
Number of comments stored: 11
i : 15
Number of comments stored: 12
i : 16
Number of comments stored: 13
i : 17
Number of comments stored: 14
i : 18
Number of comments stored: 15
i : 19
Number of comments stored: 16
i : 20
Number of comments stored: 17
i : 21
Number of comments stored: 18
i : 22
Number of comments stored: 19
i : 23
Number of comments stored: 20
i : 24
Number of comments stored: 21
i : 25
Number of comments stored: 22
i : 26
Number of comments stored: 23
i : 27
Number of comments stored: 24
i : 28
Number of comments stored: 25
i : 29
Number of comments stored: 26
i : 30
Number of comments stored: 27

Number of comments stored: 216
i : 220
Number of comments stored: 217
i : 221
Number of comments stored: 218
i : 222
Number of comments stored: 219
i : 223
Number of comments stored: 220
i : 224
Number of comments stored: 221
i : 225
Number of comments stored: 222
i : 226
Number of comments stored: 223
i : 227
Number of comments stored: 224
i : 228
Number of comments stored: 225
i : 229
Number of comments stored: 226
i : 230
Number of comments stored: 227
i : 231
Number of comments stored: 228
i : 232
Number of comments stored: 229
i : 233
Number of comments stored: 230
i : 234
Number of comments stored: 231
i : 235
Number of comments stored: 232
i : 236
Number of comments stored: 233
i : 237
Number of comments stored: 234
i : 238
Number of comments stored: 235
i : 239
Number of comments stored: 236
i : 240
Number of comments stored: 237
i : 241
Number of comments stored: 238
i : 242
Number of comments stored: 239
i : 243
Number of comments stored: 240
i : 244
Number of comments stored

Number of comments stored: 427
i : 431
Number of comments stored: 428
i : 432
Number of comments stored: 429
i : 433
Number of comments stored: 430
i : 434
Number of comments stored: 431
i : 435
Number of comments stored: 432
i : 436
Number of comments stored: 433
i : 437
Number of comments stored: 434
i : 438
Number of comments stored: 435
i : 439
Number of comments stored: 436
i : 440
Number of comments stored: 437
i : 441
Number of comments stored: 438
i : 442
Number of comments stored: 439
i : 443
Number of comments stored: 440
i : 444
Number of comments stored: 441
i : 445
Number of comments stored: 442
i : 446
Number of comments stored: 443
i : 447
Number of comments stored: 444
i : 448
Number of comments stored: 445
i : 449
Number of comments stored: 446
i : 450
Number of comments stored: 447
i : 451
Number of comments stored: 448
i : 452
Number of comments stored: 449
i : 453
Number of comments stored: 450
i : 454
Number of comments stored: 451
i : 455
Number of comments stored

Number of comments stored: 638
i : 642
Number of comments stored: 639
i : 643
Number of comments stored: 640
i : 644
Number of comments stored: 641
i : 645
Number of comments stored: 642
i : 646
Number of comments stored: 643
i : 647
Number of comments stored: 644
i : 648
Number of comments stored: 645
i : 649
Number of comments stored: 646
i : 650
Number of comments stored: 647
i : 651
Number of comments stored: 648
i : 652
Number of comments stored: 649
i : 653
Number of comments stored: 650
i : 654
Number of comments stored: 651
i : 655
Number of comments stored: 652
i : 656
Number of comments stored: 653
i : 657
Number of comments stored: 654
i : 658
Number of comments stored: 655
i : 659
Number of comments stored: 656
i : 660
Number of comments stored: 657
i : 661
Number of comments stored: 658
i : 662
Number of comments stored: 659
i : 663
Number of comments stored: 660
i : 664
Number of comments stored: 661
i : 665
Number of comments stored: 662
i : 666
Number of comments stored

Number of comments stored: 849
i : 853
Number of comments stored: 850
i : 854
Number of comments stored: 851
i : 855
Number of comments stored: 852
i : 856
Number of comments stored: 853
i : 857
Number of comments stored: 854
i : 858
Number of comments stored: 855
i : 859
Number of comments stored: 856
i : 860
Number of comments stored: 857
i : 861
Number of comments stored: 858
i : 862
Number of comments stored: 859
i : 863
Number of comments stored: 860
i : 864
Number of comments stored: 861
i : 865
Number of comments stored: 862
i : 866
Number of comments stored: 863
i : 867
Number of comments stored: 864
i : 868
Number of comments stored: 865
i : 869
Number of comments stored: 866
i : 870
Number of comments stored: 867
i : 871
Number of comments stored: 868
i : 872
Number of comments stored: 869
i : 873
Number of comments stored: 870
i : 874
Number of comments stored: 871
i : 875
Number of comments stored: 872
i : 876
Number of comments stored: 873
i : 877
Number of comments stored

Number of comments stored: 1057
i : 1061
Number of comments stored: 1058
i : 1062
Number of comments stored: 1059
i : 1063
Number of comments stored: 1060
i : 1064
Number of comments stored: 1061
i : 1065
Number of comments stored: 1062
i : 1066
Number of comments stored: 1063
i : 1067
Number of comments stored: 1064
i : 1068
Number of comments stored: 1065
i : 1069
Number of comments stored: 1066
i : 1070
Number of comments stored: 1067
i : 1071
Number of comments stored: 1068
i : 1072
Number of comments stored: 1069
i : 1073
Number of comments stored: 1070
i : 1074
Number of comments stored: 1071
i : 1075
Number of comments stored: 1072
i : 1076
Number of comments stored: 1073
i : 1077
Number of comments stored: 1074
i : 1078
Number of comments stored: 1075
i : 1079
Number of comments stored: 1076
i : 1080
Number of comments stored: 1077
i : 1081
Number of comments stored: 1078
i : 1082
Number of comments stored: 1079
i : 1083
Number of comments stored: 1080
i : 1084
Number of commen

Number of comments stored: 1257
i : 1261
Number of comments stored: 1258
i : 1262
Number of comments stored: 1259
i : 1263
Number of comments stored: 1260
i : 1264
Number of comments stored: 1261
i : 1265
Number of comments stored: 1262
i : 1266
Number of comments stored: 1263
i : 1267
Number of comments stored: 1264
i : 1268
Number of comments stored: 1265
i : 1269
Number of comments stored: 1266
i : 1270
Number of comments stored: 1267
i : 1271
Number of comments stored: 1268
i : 1272
Number of comments stored: 1269
i : 1273
Number of comments stored: 1270
i : 1274
Number of comments stored: 1271
i : 1275
Number of comments stored: 1272
i : 1276
Number of comments stored: 1273
i : 1277
Number of comments stored: 1274
i : 1278
Number of comments stored: 1275
i : 1279
Number of comments stored: 1276
i : 1280
Number of comments stored: 1277
i : 1281
Number of comments stored: 1278
i : 1282
Number of comments stored: 1279
i : 1283
Number of comments stored: 1280
i : 1284
Number of commen

In [49]:
csvpath = "C:/Users/dogus/Dropbox/DgsPy_DBOX/Hepsiburada/comments_youtube/"
def combine_csv_files(csvpath):
    import re
    import os
    import csv
    import pandas as pd
    os.chdir(csvpath)
    filenames = os.listdir()
    print(filenames)
    ##x = pd.read_csv('HBV0000079GT1_2.csv', header=None)
    ##print(x)
    comb = pd.concat( [pd.read_csv(f) for f in filenames])
    comb.drop_duplicates(inplace=True)
    comb.to_csv('spam_youtube2.csv', index=False)
    return comb

#df = combine_csv_files(csvpath)

['1281.csv', '1978.csv', '2086.csv', '34.csv', '4413.csv', '5801.csv', '6639.csv', '6973.csv', '7966.csv', '7987.csv', '8187.csv', '8235.csv', '824.csv', '8325.csv', '8716.csv', '8981.csv', '9151.csv', '9354.csv']


In [51]:
#
#df.describe()

,Name,Date,Comment
count,15509,15509,15509
unique,13770,197,15225
top,Huseyin Yurdakul,1 yıl önce,"['Firit', 'YANITLA']"
freq,243,2311,21


In [52]:
#df.to_csv('spam_youtube1.csv')

In [13]:
#driver.save_screenshot('asdasd.png')

ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine

In [ ]:
#'https://www.youtube.com/watch?v=U2rdt8PEKi4'
#https://www.youtube.com/watch?v=1ZNrst0sKuE
#"https://www.youtube.com/watch?v=AtNX2bZGgow"
#https://www.youtube.com/watch?v=2Lpc4GSJ9_M
#https://www.youtube.com/watch?v=GbKofBqKtCg
#https://www.youtube.com/watch?v=Yu_nUa9wUg4
#https://www.youtube.com/watch?v=dSR1zLev73I
#https://www.youtube.com/watch?v=ZcXMLNsjVwM
#https://www.youtube.com/watch?v=Keyd6yeedgo
#https://www.youtube.com/watch?v=xAYGOGghYp0
#https://www.youtube.com/watch?v=eOuQCwnSNJQ
#https://www.youtube.com/watch?v=jzVhWwVimNM
#https://www.youtube.com/watch?v=-eIdh-I147Q
#https://www.youtube.com/watch?v=KduZT_HY3WI
#https://www.youtube.com/watch?v=dUlBTU1J_e8
#https://www.youtube.com/watch?v=lJaP6YGRo14
#https://www.youtube.com/watch?v=Y7s-kPT13rg
#https://www.youtube.com/watch?v=AccyIlzp_t0
#https://www.youtube.com/watch?v=NlrT7j8oHXY
#
#
#